In [211]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [212]:
# Importing required libraries
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import f1_score
# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

## Import data

In [213]:
#13k top spotify songs 
#df=pd.read_csv('data/df.csv', sep=",")
#users favorite tracks based on time 
df_fav=pd.read_csv('data/df_fav.csv', sep=",")
df=pd.read_csv('data/df_hot.csv', sep=",")

In [214]:
df.head()

,track_id,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
0,3rUGC1vUpkDG9CZFHMur1t,131872,7016,9646,1,0.750,0.2560,0.733,111.018,0.000000,...,0.844,0.0319,0,98.0,2.197867,50.511708,4.5,111.018,0.000000,0
1,7aqfrAY2p9BUSiupwk3svU,247444,2075,2891,4,0.470,0.0277,0.640,163.920,0.000000,...,0.248,0.3200,1,91.0,4.124067,39.747175,49.5,655.680,0.000000,0
2,7dJYggqjKo71KI9sLzqCs8,265360,2007,7025,4,0.750,0.2280,0.674,123.026,0.000089,...,0.392,0.0970,0,89.0,4.422667,27.817154,89.5,492.104,0.000391,0
3,4MjDJD8cW7iVeWInc2Bdyj,267194,742,9691,4,0.787,0.1500,0.621,139.056,0.000402,...,0.130,0.0680,0,94.0,4.453233,31.225851,21.0,556.224,0.002680,0
4,5aIVCx5tnk0ntmdiinnYvw,200256,7579,9135,4,0.673,0.0856,0.722,117.187,0.000000,...,0.519,0.0755,0,92.0,3.337600,35.111158,37.5,468.748,0.000000,0


In [215]:
df.describe()

,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
count,1.259400e+04,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,...,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000,12594.000000
mean,2.385551e+05,4135.184215,4870.247816,4.502938,0.553316,0.315991,0.592348,119.010716,0.182240,-9.627065,...,0.469516,0.084973,0.672622,50.297933,3.975918,35.102960,6180.605606,465.286804,537.973461,0.009370
std,1.263497e+05,2385.110827,2834.519720,6.384068,0.189292,0.361614,0.283803,31.161578,0.322545,6.945035,...,0.428565,0.110423,0.684204,19.924747,2.105828,18.796769,3633.365412,132.168642,11978.910413,0.096346
min,9.750000e+03,0.000000,0.000000,0.000000,0.000000,0.000001,0.000568,0.000000,0.000000,-44.150000,...,0.000000,0.000000,0.000000,0.000000,0.162500,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.771040e+05,2094.250000,2406.000000,4.000000,0.428000,0.016200,0.413000,97.484000,0.000000,-10.962500,...,0.219000,0.036700,0.000000,39.000000,2.951733,24.422204,3045.500000,380.037000,0.000000,0.000000
50%,2.119465e+05,4201.500000,4908.500000,4.000000,0.571000,0.128000,0.655000,121.940000,0.000332,-7.214500,...,0.423000,0.047400,1.000000,52.000000,3.532442,33.464736,6092.000000,480.024000,0.005413,0.000000
75%,2.592970e+05,6213.750000,7320.750000,4.000000,0.693000,0.606750,0.827000,137.689500,0.180000,-5.198000,...,0.644000,0.079600,1.000000,65.000000,4.321617,42.873465,9370.500000,543.818000,0.922067,0.000000
max,4.170227e+06,8136.000000,9764.000000,96.000000,0.978000,0.996000,0.999000,219.446000,0.986000,1.177000,...,5.000000,1.000000,11.000000,100.000000,69.503783,574.662857,12343.000000,926.355000,747863.247863,1.000000


In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12594 entries, 0 to 12593
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   track_id                     12594 non-null  object 
 1   length                       12594 non-null  int64  
 2   artists                      12594 non-null  int64  
 3   album                        12594 non-null  int64  
 4   time_signature               12594 non-null  int64  
 5   danceability                 12594 non-null  float64
 6   acousticness                 12594 non-null  float64
 7   energy                       12594 non-null  float64
 8   tempo                        12594 non-null  float64
 9   instrumentalness             12594 non-null  float64
 10  loudness                     12594 non-null  float64
 11  liveness                     12594 non-null  float64
 12  key                          12594 non-null  float64
 13  valence         

In [217]:
df['favorite'].value_counts()

favorite
0    12476
1      118
Name: count, dtype: int64

In [218]:
print(df.shape)

(12594, 23)


In [219]:
df.head()

,track_id,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
0,3rUGC1vUpkDG9CZFHMur1t,131872,7016,9646,1,0.750,0.2560,0.733,111.018,0.000000,...,0.844,0.0319,0,98.0,2.197867,50.511708,4.5,111.018,0.000000,0
1,7aqfrAY2p9BUSiupwk3svU,247444,2075,2891,4,0.470,0.0277,0.640,163.920,0.000000,...,0.248,0.3200,1,91.0,4.124067,39.747175,49.5,655.680,0.000000,0
2,7dJYggqjKo71KI9sLzqCs8,265360,2007,7025,4,0.750,0.2280,0.674,123.026,0.000089,...,0.392,0.0970,0,89.0,4.422667,27.817154,89.5,492.104,0.000391,0
3,4MjDJD8cW7iVeWInc2Bdyj,267194,742,9691,4,0.787,0.1500,0.621,139.056,0.000402,...,0.130,0.0680,0,94.0,4.453233,31.225851,21.0,556.224,0.002680,0
4,5aIVCx5tnk0ntmdiinnYvw,200256,7579,9135,4,0.673,0.0856,0.722,117.187,0.000000,...,0.519,0.0755,0,92.0,3.337600,35.111158,37.5,468.748,0.000000,0


In [220]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


# Assuming df is your DataFrame

# Drop non-essential columns and split the data into features (X) and target variable (y)
#X = df.drop(columns=['favorite', 'track_id'])  # Drop 'favorite' and 'track_id' columns
X = df[['valence', 'key', 'time_signature','artists','tempo','beat_density','speechiness','melody_complexity']]

y = df['favorite']  # Select the 'favorite' column as the target variable

In [221]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [222]:
X_train = pd.DataFrame(X_train, columns=['valence', 'key', 'time_signature','artists','tempo','beat_density','speechiness','melody_complexity'])
X_test = pd.DataFrame(X_test, columns=['valence', 'key', 'time_signature','artists','tempo','beat_density','speechiness','melody_complexity'])

X_train = pd.DataFrame(X_train, columns=['length', 'artists', 'album', 'time_signature', 'danceability',
       'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness',
       'liveness', 'key', 'valence', 'speechiness', 'mode', 'Popularity',
       'minutes', 'beat_density', 'Popularity_Rank', 'melody_complexity',
       'instrumental_acoustic_ratio'])
X_test = pd.DataFrame(X_test, columns=['length', 'artists', 'album', 'time_signature', 'danceability',
       'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness',
       'liveness', 'key', 'valence', 'speechiness', 'mode', 'Popularity',
       'minutes', 'beat_density', 'Popularity_Rank', 'melody_complexity',
       'instrumental_acoustic_ratio'])

In [223]:
from imblearn.over_sampling import SMOTE
# Train / Split Data
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X, y) 

In [224]:
y_train.value_counts()

favorite
0    12476
1    12476
Name: count, dtype: int64

In [225]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(24952, 8)
(24952,)
(2519, 8)
(2519,)


In [206]:
# Initialize the KNN model and fit it to the training data
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = knn.predict(X_test)


In [207]:
# Evaluate the model (you can use classification metrics to assess its performance)
# For instance, you can use accuracy_score, classification_report, confusion_matrix, etc.
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Further evaluation
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.007939658594680429
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2499
           1       0.01      1.00      0.02        20

    accuracy                           0.01      2519
   macro avg       0.00      0.50      0.01      2519
weighted avg       0.00      0.01      0.00      2519

[[   0 2499]
 [   0   20]]


C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1471: Unde

In [208]:
from sklearn.model_selection import cross_val_score, KFold

# Assuming X and y are your features and target variable

# Initialize the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors

# Define the number of folds for cross-validation
num_folds = 5

# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(knn, X, y, cv=kf)

# Output the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean Accuracy: 1.0


In [209]:
from sklearn.metrics import confusion_matrix
# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[   0 2499]
 [   0   20]]


In [226]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler


#random forest 
# Initialize the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters like n_estimators, max_depth, etc.

# Fit the Random Forest model to the training data
rf_classifier.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Display classification report
print(classification_report(y_test, y_pred))

# Generate and display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2499
           1       1.00      1.00      1.00        20

    accuracy                           1.00      2519
   macro avg       1.00      1.00      1.00      2519
weighted avg       1.00      1.00      1.00      2519

Confusion Matrix:
[[2499    0]
 [   0   20]]
